In [25]:
# binary classification, breast cancer dataset, label and one hot encoded
import joblib
import numpy
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# load data
data = read_csv('breast_cancer.csv', header=None)
dataset = data.values

# split data into X and y
X = dataset[:,0:9]
X = X.astype(str)
Y = dataset[:,9]

# encode string input values as integers
encoded_x = None
for i in range(0, X.shape[1]):
	label_encoder = LabelEncoder()
	label_encoder.fit(X[:,i])
	joblib.dump(label_encoder, f'model_dir/label-encoder-{i}.joblib')
	feature = label_encoder.transform(X[:,i])
	feature = feature.reshape(X.shape[0], 1)
	onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
	onehot_encoder = onehot_encoder.fit(feature)
	joblib.dump(onehot_encoder, f'model_dir/encoder-{i}.joblib')
	feature = onehot_encoder.transform(feature)
	if encoded_x is None:
		encoded_x = feature
	else:
		encoded_x = numpy.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)

# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
joblib.dump(label_encoder, 'model_dir/output-label-encoder.joblib')
label_encoded_y = label_encoder.transform(Y)

# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(encoded_x, label_encoded_y, test_size=test_size, random_state=seed)

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

X shape: :  (286, 43)
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)
Accuracy: 68.42%


## Save the model and load it back into memory

In [17]:
model.save_model('model_dir/xgboost.txt')

## Import flighty SDK and initialize

In [1]:
import flighty as f

# f.set_hostname('127.0.0.1:8002')

Initializing flighty...
    You are running flighty pointing at the default hostname of http://127.0.0.1/api/v1
    To point at your own hostname please run flighty.set_hostname(<your_hostname>)


In [9]:
f.create_endpoint(name='doc-rec')
f.create_artifact(path='./model_dir', name='xgboost', version=0)
f.create_handler(endpoint='doc-rec', name='xgboost', 
  docker_image='docker.io/gvashishtha/flighty:flighty-demo',
  code_path='./code_sample', artifacts=[{'name': 'xgboost', 'version': 0}])

{'name': 'doc-rec', 'url': 'http://127.0.0.1/doc-rec'}
{'name': 'xgboost',
 'path': '/code/flighty-files/xgboost/0',
 'type': 'model',
 'version': 0}
{'name': 'doc-rec-xgboost-0-code',
 'path': '/code/flighty-files/doc-rec-xgboost-0-code/0',
 'type': 'code',
 'version': 0}
{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
 'endpoint': 'doc-rec',
 'name': 'xgboost',
 'version': 0}


In [11]:
response = f.invoke(endpoint='doc-rec', body={"data": {
    "age": "40-49",
    "meno": "lt40",
    "tumor_size": "0-4",
    "inv_nodes": "0-2",
    "node_caps": "yes",
    "deg_malig": "1",
    "breast": "left",
    "breastquad": "left_up",
    "irradiat": "yes"
  }})
print(response.text, response.status_code)

"Your prediction is ['no-recurrence-events']" 200


In [12]:
f.list_handlers()

[{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
  'endpoint_id': 2,
  'id': 2,
  'name': 'xgboost',
  'prod_traffic': 100,
  'shadow_traffic': 0,
  'version': 0}]


In [13]:
f.delete_handler(endpoint='doc-rec', name='xgboost', version=0)
f.delete_endpoint(name='doc-rec')
f.delete_artifact(name='xgboost', version=0)

None
'Artifact doc-rec-xgboost-0-code with version 0 successfully deleted.'
'Endpoint doc-rec successfully deleted.'
'Artifact xgboost with version 0 successfully deleted.'


In [8]:
f.list_artifacts()
f.list_endpoints()
f.list_handlers()

[]
[]
[]
